In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised = True, with_info = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteOGBSE8/fashion_mnist-train.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteOGBSE8/fashion_mnist-test.tfrecord*...:   0%…

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [4]:
class_names = metadata.features['label'].names
print(class_names)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [5]:
train_num = metadata.splits['train'].num_examples
test_num = metadata.splits['test'].num_examples
print(train_num, test_num)

60000 10000


In [6]:
def normalise(image, labels):
    image = tf.cast(image, tf.float32)
    image/=255
    return image, labels
train_dataset = train_dataset.map(normalise)
test_dataset = test_dataset.map(normalise)
train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), padding='same', activation = tf.nn.relu, input_shape=(28,28,1)),
    tf.keras.layers.MaxPool2D((2,2), strides = 2),
    tf.keras.layers.Conv2D(64,(3,3), padding = 'same', activation = tf.nn.relu),
    tf.keras.layers.MaxPool2D((2,2),strides = 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [10]:
model.compile(optimizer = 'Adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [11]:
batch = 32
train_dataset = train_dataset.cache().repeat().shuffle(train_num).batch(batch)
test_dataset = test_dataset.cache().batch(batch)


In [12]:
model.fit(train_dataset, epochs = 10, steps_per_epoch=math.ceil(train_num/batch))


Epoch 1/10
1875/1875 [==============================] - 21s 4ms/step - loss: 0.4085 - accuracy: 0.8535
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2654 - accuracy: 0.9036
Epoch 3/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2182 - accuracy: 0.9199
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1890 - accuracy: 0.9311
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1580 - accuracy: 0.9416
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1361 - accuracy: 0.9499
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1167 - accuracy: 0.9569
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1004 - accuracy: 0.9629
Epoch 9/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0848 - accuracy: 0.9679
Epoch 10/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0

In [13]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps = math.ceil(test_num/batch))

313/313 [==============================] - 2s 5ms/step - loss: 0.3414 - accuracy: 0.9117


In [18]:
for test_image, test_label in test_dataset.take(1):
  test_image = test_image.numpy()
  test_label = test_label.numpy()
  prediction = model.predict(test_image)

print(prediction[0])

1/1 [==============================] - 0s 15ms/step
[8.63953389e-08 5.82228753e-12 3.03374091e-03 1.01754556e-10
 8.84593010e-01 4.87525611e-08 1.12373158e-01 6.60508981e-10
 3.36616623e-09 1.64843028e-09]


In [19]:
print(prediction[1])

[1.0518616e-09 2.2664719e-15 5.6734632e-05 6.9595985e-10 6.3794547e-01
 1.8279498e-07 3.6199763e-01 1.5934806e-10 5.3888192e-12 6.5917194e-13]


In [20]:
np.argmax(prediction[0])

4